<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# DSI-SG-42 Project 4:
###
---

## 4. Modeling

### 4.1 Importing Libraries

In [194]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb
import time

# For graphs and visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# Essential library imports for Modeling Pre-Processing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Library to load the Models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For Modeling Pipeline & Hyperparameter Tuning (GridSearchCV)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# For Modeling Metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import roc_curve, auc

# For Deep Learning Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

### 4.2 Import cleaned dataset

In [195]:
df = pd.read_csv('sample_data/final_dataset.csv')
df.head()

,height,weight,bmi,yrssmok,packday,sleep_hours,age,health_status,phys_health_not_good,mental_health_not_good,...,asthma_status,race_ethnicity,sex,education,income,smoker_status,e_cig_smoker,binge_drinker,heavy_drinker,chd_mi
0,1.744,81.376,28.054,0.0,0.0,8.0,80.0,2.0,1.0,1.0,...,3.0,1.0,2.0,4.0,7.0,4.0,1.0,1.0,1.0,2.0
1,1.600,68.040,26.580,0.0,0.0,6.0,80.0,1.0,1.0,1.0,...,3.0,1.0,2.0,2.0,5.0,4.0,1.0,1.0,1.0,2.0
2,1.570,63.500,25.760,0.0,0.0,5.0,56.0,2.0,2.0,2.0,...,3.0,1.0,2.0,4.0,10.0,4.0,1.0,1.0,1.0,2.0
3,1.650,63.500,23.320,56.0,0.1,7.0,73.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,7.0,2.0,1.0,1.0,1.0,2.0
4,1.570,53.980,21.900,0.0,0.0,9.0,43.0,4.0,2.0,1.0,...,3.0,1.0,2.0,3.0,5.0,4.0,1.0,1.0,1.0,2.0


### 4.3 Checking the dataset

#### 4.3.1 Checking columns and datatypes

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440111 entries, 0 to 440110
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   height                     440111 non-null  float64
 1   weight                     440111 non-null  float64
 2   bmi                        440111 non-null  float64
 3   yrssmok                    440111 non-null  float64
 4   packday                    440111 non-null  float64
 5   sleep_hours                440111 non-null  float64
 6   age                        440111 non-null  float64
 7   health_status              440111 non-null  float64
 8   phys_health_not_good       440111 non-null  float64
 9   mental_health_not_good     440111 non-null  float64
 10  last_routine_checkup       440111 non-null  float64
 11  visit_dentist_past_year    440111 non-null  float64
 12  health_insurance           440111 non-null  float64
 13  phy_exercise_past_30_days  44

#### 4.3.2 Check for imbalanced dataset (using target variable - `chd_mi`)

In [197]:
percentage_responses = df['chd_mi'].value_counts(normalize=True) * 100

print(percentage_responses)

chd_mi
2.0    90.96796
1.0     9.03204
Name: proportion, dtype: float64


**Analysis:**

* We checked that there are no null values - all null values have been rectified at the data cleaning stage.
* There is a imbalance in the dataset - with 90.96% of respondents reported not having any heart disease or myocardial infarctions, while, while the remaining 9.03% reported having heart disease or myocardial infarctions. We will rectify the imbalance during the when fitting the model into the pipeline.

### 4.4 Modeling

#### 4.4.1 Baseline Models

Prior to modeling, the null values have been rectified in the Data Cleaning stage. For the purpose of this project, we will run look into 4 Classifier-type models as a baseline.

- **Logistic Regression**
  - Simplicity and Interpretability: As a straightforward algorithm, it serves as a good starting point for binary classification tasks. It provides a probabilistic framework which means that besides making predictions, it can also quantify the uncertainty of its predictions, which is useful for understanding the impact of each feature on the prediction.
  - Performance: Despite its simplicity, Logistic Regression can perform quite well on linearly separable data.
  - Speed: It's computationally inexpensive, making it fast for both training and prediction, which is beneficial when working with very large datasets.
  - Baseline Comparison: Commonly used as a benchmark because of the ease to implement and interpret.

- **XGBoost**
  - Accuracy: XGBoost is known for its high performance and speed in classification problems, using a gradient boosting framework.
  - Flexibility: XGBoost allows users to define custom optimization objectives and evaluation criteria, adding a layer of sophistication to the modeling.

- **Random Forest**
  - Versatility: Random Forest performs well on a wide range of data types without the need for extensive data preprocessing like scaling and normalization.
  - Robustness: As an ensemble method, it is less prone to overfitting than a single decision tree and often has a very good performance right out of the box.
  - Feature Importance: It provides a straightforward indication of feature importance based on how much they contribute to reducing variance, which is helpful for feature selection.

- **Decision Trees**
  
  - Non-Parametric: As a non-parametric method, it makes no assumptions about the underlying distributions of the data, which is useful for practical applications.
  - Handling Non-Linear Relationships: It can capture non-linear relationships between features and the target variable.



##### 4.4.1.1 Logistic Regression

In [198]:
# Separate features and target from the cleaned dataframe
log_X = df.drop('chd_mi', axis=1)
log_y = df['chd_mi'].astype(int).map({1: 0, 2: 1})  # Map values and ensure int type

# Encoding categorical variables
log_X_encoded = pd.get_dummies(log_X)

# Ensuring log_X and log_y have consistent lengths
assert len(log_X_encoded) == len(log_y), "log_X and log_y have inconsistent number of samples."

# Perform the train-test split
log_X_train, log_X_test, log_y_train, log_y_test = train_test_split(log_X_encoded, log_y, test_size=0.2, stratify=log_y, random_state=42)

In [199]:
# Feature scaling
scaler = StandardScaler()
log_X_train_scaled = scaler.fit_transform(log_X_train)
log_X_test_scaled = scaler.transform(log_X_test)

In [200]:
# Initialize the Logistic Regression model with a higher max_iter
model = LogisticRegression(max_iter=5000, random_state=42)

# Train the model on the scaled data
model.fit(log_X_train_scaled, log_y_train)

# Make predictions using the correctly scaled data
log_train_pred = model.predict(log_X_train_scaled)
log_test_pred = model.predict(log_X_test_scaled)

# Calculate and print training and test accuracies
log_train_accuracy = accuracy_score(log_y_train, log_train_pred)
log_test_accuracy = accuracy_score(log_y_test, log_test_pred)
print(f"LogReg Train Accuracy: {log_train_accuracy * 100:.2f}%")
print(f"LogReg Test Accuracy: {log_test_accuracy * 100:.2f}%")

# For cross-validation, ensure data is scaled before the process
log_cv_scores = cross_val_score(model, scaler.fit_transform(log_X_encoded), log_y, cv=5, scoring='accuracy')

# Print the average of the cross-validation scores and scores for each fold
print(f"LogReg Cross-Validation: {log_cv_scores.mean() * 100:.2f}%")
print(f"LogReg Cross-Validation (per Fold): {[f'{score * 100:.2f}%' for score in log_cv_scores]}")

LogReg Train Accuracy: 91.10%
LogReg Test Accuracy: 91.12%
LogReg Cross-Validation: 91.09%
LogReg Cross-Validation (per Fold): ['91.05%', '91.07%', '91.10%', '91.09%', '91.13%']


##### 4.4.1.2 XGBoost

In [201]:
pip install xgboost

In [202]:
# Separate features and target
xg_X = df.drop('chd_mi', axis=1)
xg_y = df['chd_mi'].astype(int)

# Map the values of xg_y from [1, 2] to [0, 1]
xg_y_mapped = xg_y.map({1: 0, 2: 1})

# Verify the consistency in the number of samples between xg_X and y_mapped
assert len(xg_X) == len(y_mapped), "The feature set xg_X and target variable y_mapped have inconsistent lengths."

# Now, you can safely perform the train-test split
xg_X_train, xg_X_test, xg_y_train, xg_y_test = train_test_split(xg_X, xg_y_mapped, test_size=0.2, stratify=xg_y_mapped, random_state=42)

In [203]:
# Initialize the XGBoost classifier with enable_categorical=True
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)

# Train the model using the correct xg_y_train variable
model.fit(xg_X_train, xg_y_train)  # Use xg_y_train directly after ensuring it's correctly mapped and split

# Make predictions on training and test sets
xg_train_pred = model.predict(xg_X_train)
xg_test_pred = model.predict(xg_X_test)

# Calculate and print training and test accuracies
# Here, make sure to use 'xg_y_train' and 'xg_y_test' which are the variables you should have defined after the train-test split and mapping
xg_train_accuracy = accuracy_score(xg_y_train, xg_train_pred)
xg_test_accuracy = accuracy_score(xg_y_test, xg_test_pred)
print(f"XGBoost Train Accuracy: {xg_train_accuracy * 100:.2f}%")
print(f"XGBoost Test Accuracy: {xg_test_accuracy * 100:.2f}%")

# Ensure X is suitable for cross-validation by converting object types to 'category' if needed
# This step might not be necessary for models like XGBoost when using enable_categorical=True
# but is kept for demonstration or if you plan to use models that do not natively support categorical features
xg_X_for_cv = xg_X.copy()
for col in xg_X_for_cv.columns:
    if xg_X_for_cv[col].dtype == 'object':
        xg_X_for_cv[col] = xg_X_for_cv[col].astype('category')

# Perform 5-fold cross-validation using the mapped y
xg_cv_scores = cross_val_score(model, xg_X_for_cv, xg_y_mapped, cv=5, scoring='accuracy')

# Print the average of the cross-validation scores and the scores for each fold
print(f"XGBoost Cross-Validation: {xg_cv_scores.mean() * 100:.2f}%")
print(f"XGBoost Cross-Validation (per Fold): {[f'{score * 100:.2f}%' for score in xg_cv_scores]}")

XGBoost Train Accuracy: 91.65%
XGBoost Test Accuracy: 91.10%
XGBoost Cross-Validation: 91.06%
XGBoost Cross-Validation (per Fold): ['91.04%', '91.07%', '91.04%', '91.02%', '91.10%']


##### 4.4.1.3 Random Forest

In [204]:
pip install scikit-learn

In [205]:
# Separate features and target
rf_X = df.drop('chd_mi', axis=1)
rf_y = df['chd_mi']

# Encoding categorical variables
rf_X_encoded = pd.get_dummies(rf_X)

# Splitting dataset into training and testing sets
rf_X_train, rf_X_test, rf_y_train, rf_y_test = train_test_split(rf_X_encoded, rf_y, test_size=0.2, stratify=rf_y, random_state=42)

In [ ]:
# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, bootstrap=True, random_state=42)

# Training the model
model.fit(rf_X_train, rf_y_train)

# Making predictions for evaluation
rf_y_train_pred = model.predict(rf_X_train)
rf_y_test_pred = model.predict(X_test)

# Calculating Train and Test accuracy scores
rf_train_accuracy = accuracy_score(rf_y_train, rf_y_train_pred)
rf_test_accuracy = accuracy_score(rf_y_test, rf_y_test_pred)
print(f"Random Forest Train Accuracy: {rf_train_accuracy * 100:.2f}%")
print(f"Random Forest Test Accuracy: {rf_test_accuracy * 100:.2f}%")

# Perform 5-fold cross-validation to evaluate the model
rf_cv_scores = cross_val_score(model, rf_X_encoded, rf_y, cv=5)

# Calculate and print the average of the cross-validation scores
cv_mean = rf_cv_scores.mean()
cv_std = rf_cv_scores.std()
print(f"Random Forest Cross-Validation: {rf_cv_scores.mean() * 100:.2f}%")
print(f"Random Forest Cross-Validation (per Fold): {[f'{score * 100:.2f}%' for score in rf_cv_scores]}")

##### 4.4.1.4 Decision Tree

In [ ]:
# Separate features and target
dt_X = df.drop('chd_mi', axis=1)
dt_y = df['chd_mi']

# Separate numerical and categorical columns
numeric_features = dt_X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = dt_X.select_dtypes(include=['object', 'category']).columns

# Scale the numerical features
scaler = StandardScaler()
dt_X_numeric_scaled = scaler.fit_transform(dt_X[numeric_features])

# One-hot encode the categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
dt_X_categorical_encoded = encoder.fit_transform(dt_X[categorical_features])

# Combine the processed numerical and categorical features back into a single dataset
dt_X_processed = np.concatenate([dt_X_numeric_scaled, dt_X_categorical_encoded], axis=1)

# Now you have a fully processed dataset, you can split it into training and testing sets
dt_X_train, dt_X_test, dt_y_train, dt_y_test = train_test_split(dt_X_processed, dt_y, test_size=0.2, stratify=dt_y, random_state=42)

In [ ]:
# Initialize the Decision Tree model
model = DecisionTreeClassifier(random_state=42)

# Train the model
model.fit(dt_X_train, dt_y_train)

# Make predictions
dt_y_train_pred = model.predict(dt_X_train)
dt_y_test_pred = model.predict(dt_X_test)

# Evaluate the model
dt_train_accuracy = accuracy_score(dt_y_train, dt_y_train_pred)
dt_test_accuracy = accuracy_score(dt_y_test, dt_y_test_pred)
print(f"Decision Trees Train Accuracy: {dt_train_accuracy * 100:.2f}%")
print(f"Decision Trees Test Accuracy: {dt_test_accuracy * 100:.2f}%")

# Preprocessing for CV score
for col in dt_X.columns:
    if dt_X[col].dtype == 'object':
        dt_X[col] = dt_X[col].astype('category')

dt_cv_scores = cross_val_score(model, dt_X_processed, dt_y, cv=5)
cv_mean = dt_cv_scores.mean()
cv_std = dt_cv_scores.std()
print(f"Decision Trees Cross-Validation: {dt_cv_scores.mean() * 100:.2f}%")
print(f"Decision Trees Cross-Validation (per Fold): {[f'{score * 100:.2f}%' for score in dt_cv_scores]}")

**Analysis:**

Albeit, the relatively high scores generally, the Logistic Regression and XGBoost models stood out in terms of:

* Consistent Performance: The model has demonstrated high accuracy that is consistent across training, testing, and cross-validation. This consistency suggests that the model generalizes well and is not overfitting the data.

* Simplicity and Interpretability: Logistic Regression’s simplicity is a considerable advantage. It’s faster to train and easier to interpret than more complex models. This is  beneficial when explaining the model's predictions and decisions to non-technical stakeholders.

* Baseline Performance: Given that Logistic Regression is often used as a baseline model, its strong performance here validates its effectiveness for this particular dataset and problem. No doubt, XGBoost has a marginally higher training accuracy compared to Logistic Regression and holds its ground on test accuracy, suggesting it can capture complex patterns in the data without overfitting.

* Robustness to Variance: Given the slightly better performance on the training set with no loss on the test set, XGBoost may be more robust to variance in the data than Logistic Regression.


To further the point, the reason why Random Forest and Decision Trees were not chosen are as follows:

* Overfitting
  * The Random Forest training accuracy compared to its test accuracy indicates that it has overfit the training data.
  * The decision tree model shows an almost perfect training score but significantly lower test accuracy, which is a clear indicator of overfitting.

* Computational Limitations: Random Forest can be more computationally expensive and time-consuming (8mins), which could be a concern for larger datasets or limited computational resources, especially when we include tuned parameters.

* Poor Generalization:
  * The drop in cross-validation mean score in the Decision Trees model further supports the poor generalization capability of the decision tree model to unseen data compared to the other models.
  * Although there is high test accuracy in Random Forest, the discrepancy suggests that the model may not generalize as well to unseen data.

#### 4.4.2 Modeling with Parameters

For the hyperparameter tuning, we will narrow in on Logistic Regression and XGBoost, and compare on our final model from there.

##### 4.4.2.1 Initializing the Pipeline

Initialize the pipelines for the LogReg and XGBoost model with the following items:

* `Scaler`: We use `StandardScaler` here. Removes the mean and scales the graph to unit variance. This ensures consistency and efficiency of the train data when modeled.
* `SMOTE`: Synthetically Oversamples the Minority Class to balance the class distributed, using a small k-nearest neighbouring technique. Most suitable for imbalanced dataset such as the one we're using here. We initially considered the use of `ADASYN` to handle imbalance, however the time required to process was >4x times that required of the `SMOTE` technique. Moreoever, the results from the 2 techniques very marginally affects the Train-Test score of the hypertuned models (0.001).
* `Model`: Including the model as the last step in the pipeline allows for seamless integration of preprocessing steps and model training, facilitating a straightforward and reproducible process for model evaluation and selection.
  * Logistic Regression: The simplicity of the binary (0 or 1), and Probability (0 to 1) output often leads to ease of reader's interpretability.
  * XGBoost: Average target value for regression tasks or the log odds for a classification. The use of XGBoost will also help to regulate overfitting at an efficient and speedy manner

In [ ]:
# Logistic Regression Pipeline
imb_logistic_pipeline = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42, sampling_strategy=0.5)),
    ('model', LogisticRegression(random_state=42))
])

# XGBoost Pipeline
imb_xgb_pipeline = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42, sampling_strategy=0.5)),
    ('model', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

##### 4.4.2.2 Parameter Grids

The following parameters have been meticulously selected to fine-tune the models for optimal performance, especially when dealing with imbalanced and also huge dataset such as ours (400+K).
[Note: The following parameters below have a `model__` as a prefix]

**1) Logistic Regression**
* `C`: Tests a range from strong to weak regularization to balance model complexity and accuracy.
* `penalty`: 'l2' helps manage multicollinearity and model generalization.
* `solver`: 'saga' is chosen for its efficiency with large data and support for 'l1' and 'l2' penalties.
* `class_weight`: 'balanced' adjusts weights inversely to class frequencies, addressing class imbalance.
* `max_iter`: Higher iterations ensure convergence for complex datasets.

**2) XGBoost**
* `max_depth` and `n_estimators`: Control model complexity, balancing between capturing patterns and preventing overfitting.
* `learning_rate`: A moderate rate for effective learning without overshooting.
* `scale_pos_weight`: Adjusts positive to negative class weights for imbalanced data handling.
* `subsample` and `colsample_bytree`: Introduce randomness to prevent overfitting by selecting fractions of samples and features.
* `gamma`: Manages the trade-off between model simplicity and accuracy.
* `reg_lambda`: Further tuning to prevent overfitting, applying reg_lambda L2 regularization helps to smoothen the weights.

In [ ]:
logistic_param_grid = {
    'model__C': [0.001, 0.01, 0.1, 1, 10],
    'model__penalty': ['l2'],
    'model__solver': ['saga'],
    'model__class_weight': ['balanced'],
    'model__max_iter': [500, 1000],
    'smote__sampling_strategy': [0.5, 0.75, 1.0],  # Trying different oversampling ratios
}

In [ ]:
# Calculate class weights if your data is imbalanced
scale_pos_weight = sum(xg_y_train == 0) / sum(xg_y_train == 1)

# Define the parameter grid
xgb_param_grid = {
    'model__max_depth': [5],
    'model__n_estimators': [100],
    'model__learning_rate': [0.1],
    'model__scale_pos_weight': [1, scale_pos_weight],  # Use both the ratio and 1
    'model__subsample': [0.7],
    'model__colsample_bytree': [0.7],
    'model__gamma': [0],
    'model__min_child_weight': [1, 5, 10],  # Important to prevent overfitting
    'model__reg_lambda': [0.5, 1, 1.5]  # Adding reg_lambda (L2 regularization term) for tuning
}

##### 4.4.2.3 Evaluating the Hypertuned Models (using GridSearchCV)

**Metrics for consideration:**
Beyond just the CV and Train-test score in evaluating the hypertuned models, we should look into F1 score and ROC AUC score as well, since there is a class imbalance in our dataset.

* F1 Score:
  - This is the harmonic mean of precision and recall, and it gives a balance between the two. In cases where there is a class imbalance, accuracy alone can be misleading; for instance, a model that predicts only the majority class will have high accuracy but poor recall. The F1 score becomes more informative because it will be low if either precision or recall is low. Thus, this captures the balance between the positive predictions and the positive actuals.
  - It's useful when you want to find a balance between the cost of false positives and false negatives.

* ROC AUC Score:
  - Aggregate measure of performance across all classification thresholds. Unlike accuracy, the AUC ROC does not get influenced by the distribution of classes.
  - Evaluates the model’s ability to discriminate between the positive and negative classes. An AUC of 0.5 suggests no discrimination (i.e., random chance), while an AUC of 1.0 represents perfect discrimination
  - Predictions are ranked rather than their absolute values.


###### 4.4.2.3.1 Logistic Regression

In [ ]:
# Separate features and target from the cleaned dataframe
loghyp_X = df.drop('chd_mi', axis=1)
loghyp_y = df['chd_mi'].astype(int).map({1: 0, 2: 1})  # Map values and ensure int type

# Encoding categorical variables
loghyp_X_encoded = pd.get_dummies(loghyp_X)

# Ensuring loghyp_X and loghyp_y have consistent lengths
assert len(loghyp_X_encoded) == len(loghyp_y), "loghyp_X and loghyp_y have inconsistent number of samples."

# Perform the train-test split
loghyp_X_train, loghyp_X_test, loghyp_y_train, loghyp_y_test = train_test_split(loghyp_X_encoded, loghyp_y, test_size=0.2, stratify=loghyp_y, random_state=42)

# Feature scaling
scaler = StandardScaler()
loghyp_X_train_scaled = scaler.fit_transform(loghyp_X_train)
loghyp_X_test_scaled = scaler.transform(loghyp_X_test)

In [ ]:
%%time
# Use StratifiedKFold for handling imbalanced datasets
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV with stratified cross-validation
logistic_grid_search = GridSearchCV(imb_logistic_pipeline, logistic_param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
logistic_grid_search.fit(loghyp_X_train, loghyp_y_train)  # Using the training data for cross-validation

# Best parameters and CV score
print("Best parameters for (hypertuned)Logistic Regression:", logistic_grid_search.best_params_)
print(f"Best CV score for (hypertuned) Logistic Regression: {logistic_grid_search.best_score_ * 100:.2f}%")

# Evaluate on training data using the best estimator found by GridSearchCV
loghyp_y_train_pred = logistic_grid_search.predict(loghyp_X_train)
loghyp_train_accuracy = accuracy_score(loghyp_y_train, loghyp_y_train_pred)
print(f"Train accuracy for (hypertuned) Logistic Regression: {loghyp_train_accuracy * 100:.2f}%")

# Evaluate on test data using the best estimator found by GridSearchCV
loghyp_y_test_pred = logistic_grid_search.predict(loghyp_X_test)
loghyp_test_accuracy = accuracy_score(loghyp_y_test, loghyp_y_test_pred)
print(f"Test accuracy for (hypertuned) Logistic Regression: {loghyp_test_accuracy * 100:.2f}%")

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
# Here's how to get the mean CV score for the best estimator across folds
loghyp_best_index = logistic_grid_search.best_index_
loghyp_mean_cv_score = logistic_grid_search.cv_results_['mean_test_score'][loghyp_best_index]
loghyp_std_cv_score = logistic_grid_search.cv_results_['std_test_score'][loghyp_best_index]
print(f"Mean CV score for the best (hypertuned) Logistic Regression model: {loghyp_mean_cv_score * 100:.2f}% ± {loghyp_std_cv_score * 100:.2f}%")

In [ ]:
# Use the best estimator to make predictions on the test set
loghyp_best_estimator = logistic_grid_search.best_estimator_
loghyp_y_test_pred = loghyp_best_estimator.predict(loghyp_X_test_scaled)
loghyp_y_test_proba = loghyp_best_estimator.predict_proba(loghyp_X_test_scaled)[:, 1]  # Get probabilities for the positive class, ensure scaled data is used

# Evaluation metrics
loghyp_conf_matrix = confusion_matrix(loghyp_y_test, loghyp_y_test_pred)
loghyp_class_report = classification_report(loghyp_y_test, loghyp_y_test_pred)
loghyp_roc_auc = roc_auc_score(loghyp_y_test, loghyp_y_test_proba)  # Use probabilities for ROC AUC score calculation
loghyp_f1 = f1_score(loghyp_y_test, loghyp_y_test_pred, average='binary')

# Extracting loghyp_TP, loghyp_TN, loghyp_FP, loghyp_FN from the confusion matrix
loghyp_TP = loghyp_conf_matrix[1, 1]
loghyp_TN = loghyp_conf_matrix[0, 0]
loghyp_FP = loghyp_conf_matrix[0, 1]
loghyp_FN = loghyp_conf_matrix[1, 0]

# Calculating Sensitivity, Specificity, Precision, and NPV
loghyp_sensitivity = loghyp_TP / (loghyp_TP + loghyp_FN)
loghyp_specificity = loghyp_TN / (loghyp_TN + loghyp_FP)
loghyp_precision = loghyp_TP / (loghyp_TP + loghyp_FP)
loghyp_npv = loghyp_TN / (loghyp_TN + loghyp_FN)

# Print metrics
print("Confusion Matrix:\n", loghyp_conf_matrix)
print("\nClassification Report:\n", loghyp_class_report)
print(f"ROC AUC Score: {loghyp_roc_auc:.4f}")
print(f"F1 Score: {loghyp_f1:.4f}")
print(f"Sensitivity: {loghyp_sensitivity:.4f}")
print(f"Specificity: {loghyp_specificity:.4f}")
print(f"Precision: {loghyp_precision:.4f}")
print(f"NPV: {loghyp_:.4f}")

In [ ]:
# Remap loghyp_y_test values from {1.0, 2.0} to {0, 1}
loghyp_y_test_binary = np.where(loghyp_y_test == 2.0, 1, 0)

# Now you can calculate FPR, TPR, and loghyp_thresholds for the ROC curve with the binary labels
loghyp_fpr, loghyp_tpr, loghyp_thresholds = roc_curve(loghyp_y_test_binary, loghyp_y_test_proba)

# Calculate the AUC (Area Under Curve)
loghyp_roc_auc_value = auc(loghyp_fpr, loghyp_tpr)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(loghyp_fpr, loghyp_tpr, color='darkorange', lw=2, label=f'ROC curve (area = {loghyp_roc_auc_value:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC: LogReg Model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
loghyp_conf_matrix = confusion_matrix(loghyp_y_test_binary, loghyp_y_test_pred, labels=[1, 0]).ravel()

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(loghyp_conf_matrix, annot=True, fmt='g', cmap='Blues',
            xticklabels=['Predicted Positive (1)', 'Predicted Negative (0)'],
            yticklabels=['Actual Positive (1)', 'Actual Negative (0)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Coronary Heart Disease or Myocardial Infarction Occurrence: LogReg Model')
plt.show()

**Analysis:**

* True Positives (TP): 65318 instances were correctly predicted as having the disease.
* True Negatives (TN): 4240 instances were correctly predicted as not having the disease.
* False Positives (FP): 3710 instances were incorrectly predicted as having the disease when they did not.
* False Negatives (FN): 14755 instances were incorrectly predicted as not having the disease when they actually did.

**Implication on Metrics:**

* Sensitivity/Recall for Positive Class (Heart Disease Present): High sensitivity [(TP)/(TP+FN)], indicating the model is good at catching positive cases.
* Specificity for Negative Class (Heart Disease Absent): Lower specificity [(TN)/(TN+FP)], indicating the model could be improved in correctly identifying those without the disease.
* Precision for Positive Class: [(TP)/(TP+FP)] is reasonably good, suggesting that when the model predicts the disease, it is often correct.
* Negative Predictive Value for Negative Class: [(TN)/(TN+FN)] indicates that there is room for improvement in predicting true negatives.
* High False Negatives: The model tends to miss a significant number of actual disease cases (high FN), which is a major concern in medical diagnosis because it means patients with the disease might not get the treatment they need.
* Accuracy: While not provided in the heatmap, it can be computed as [(TP+TN)/(TP+FP+FN+TN)]. The model appears to have a higher accuracy; however, due to the imbalance highlighted by the high FN, accuracy might not be the best standalone metric to rely upon.
* Precision-Recall Trade-off: It might be necessary to adjust the threshold for classification to improve either precision or recall, depending on which is more critical for the medical outcome. In medical diagnostics, improving recall (reducing FN) might be prioritized even if it comes at the cost of precision (increasing FP).



###### 4.4.2.3.2 XGBoost

In [ ]:
# Separate features and target
xghyp_X = df.drop('chd_mi', axghyp_xis=1)
xghyp_y = df['chd_mi'].astype(int)

# Map the values of xghyp_y from [1, 2] to [0, 1]
xghyp_y_mapped = xghyp_y.map({1: 0, 2: 1})

# Verify the consistency in the number of samples between xghyp_X and xghyp_y_mapped
assert len(xghyp_X) == len(xghyp_y_mapped), "The feature set xghyp_X and target variable xghyp_y_mapped have inconsistent lengths."

# Now, you can safely perform the train-test split
xghyp_X_train, xghyp_X_test, xghyp_y_train, xghyp_y_test = train_test_split(xghyp_X, xghyp_y_mapped, test_size=0.2, stratify=xghyp_y_mapped, random_state=42)

In [ ]:
%%time

# XGBoost GridSearchCV
xgb_grid_search = GridSearchCV(imb_xgb_pipeline, xgb_param_grid, cv=5, scoring='accuracy')
xgb_grid_search.fit(xghyp_X_train, xghyp_y_train)

# Best parameters and score
print("Best parameters for (hypertuned) XGBoost:", xgb_grid_search.best_params_)
print(f"Best score for (hypertuned) XGBoost: {xgb_grid_search.best_score_ * 100:.2f}%")

# Evaluate on training data using the best estimator found by GridSearchCV
xghyp_y_train_pred = xgb_grid_search.predict(xghyp_X_train)
xghyp_train_accuracy = accuracy_score(xghyp_y_train, xghyp_y_train_pred)
print(f"Train accuracy for (hypertuned) XGBoost: {xghyp_train_accuracy * 100:.2f}%")

# Evaluate on test data using the best estimator found by GridSearchCV
xghyp_y_test_pred = xgb_grid_search.predict(xghyp_X_test)
xghyp_test_accuracy = accuracy_score(xghyp_y_test, xghyp_y_test_pred)
print(f"Test accuracy for (hypertuned) XGBoost: {xghyp_test_accuracy * 100:.2f}%")

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
xghyp_best_index = xgb_grid_search.best_index_
xghyp_mean_cv_score = xgb_grid_search.cv_results_['mean_test_score'][xghyp_best_index]
xghyp_std_cv_score = xgb_grid_search.cv_results_['std_test_score'][xghyp_best_index]
print(f"Mean CV score for the best (hypertuned) XGBoost model: {xghyp_mean_cv_score * 100:.2f}% ± {xghyp_std_cv_score * 100:.2f}%")

In [ ]:
# Feature scaling
scaler = StandardScaler()
xghyp_X_train_scaled = scaler.fit_transform(xghyp_X_train)
xghyp_X_test_scaled = scaler.transform(xghyp_X_test)

# Use the best estimator to make predictions on the test set
xghyp_best_estimator = xgb_grid_search.best_estimator_
xghyp_y_test_pred = xghyp_best_estimator.predict(xghyp_X_test_scaled)  # Ensure xghyp_X_test is scaled
xghyp_y_test_proba = xghyp_best_estimator.predict_proba(xghyp_X_test)[:,1]  # Get probabilities for positive class

# Evaluation metrics
xghyp_conf_matrix = confusion_matrix(xghyp_y_test, xghyp_y_test_pred)
xghyp_class_report = classification_report(xghyp_y_test, xghyp_y_test_pred)
xghyp_roc_auc = roc_auc_score(xghyp_y_test, xghyp_y_test_pred)
xghyp_f1 = f1_score(xghyp_y_test, xghyp_y_test_pred)

# Extracting xghyp_TP, xghyp_TN, xghyp_FP, xghyp_FN from the confusion matrix
xghyp_TP = xghyp_conf_matrix[1, 1]
xghyp_TN = xghyp_conf_matrix[0, 0]
xghyp_FP = xghyp_conf_matrix[0, 1]
xghyp_FN = xghyp_conf_matrix[1, 0]

# Calculating Sensitivity, Specificity, Precision, and NPV
xghyp_sensitivity = xghyp_TP / (xghyp_TP + xghyp_FN)
xghyp_specificity = xghyp_TN / (xghyp_TN + xghyp_FP)
xghyp_precision = xghyp_TP / (xghyp_TP + xghyp_FP)
xghyp_npv = xghyp_TN / (xghyp_TN + xghyp_FN)

# Print metrics
print("Confusion Matrix:\n", xghyp_conf_matrix)
print("\nClassification Report:\n", xghyp_class_report)
print("ROC AUC Score:", xghyp_roc_auc)
print("F1 Score:", xghyp_f1)
print(f"Sensitivity: {xghyp_sensitivity:.4f}")
print(f"Specificity: {xghyp_specificity:.4f}")
print(f"Precision: {xghyp_precision:.4f}")
print(f"NPV: {xghyp_npv:.4f}")

In [ ]:
xghyp_y_test_pred = xghyp_best_estimator.predict(xghyp_X_test_scaled)  # Predictions are made here
xghyp_y_test_proba = xghyp_best_estimator.predict_proba(xghyp_X_test_scaled)[:, 1]  # Probabilities for the positive class

# Calculate FPR, TPR, and threshold values
xghyp_fpr, xghyp_tpr, xghyp_thresholds = roc_curve(xghyp_y_test, xghyp_y_test_proba)

# Calculate the AUC (Area Under the Curve)
xghyp_roc_auc = auc(xghyp_fpr, xghyp_tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(xghyp_fpr, xghyp_tpr, color='darkorange', lw=2, label=f'ROC curve (area = {xghyp_roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal 45 degree line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: XGBoost Model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Assuming '1' in your target variable y represents the presence of the disease
xghyp_conf_matrix = confusion_matrix(xghyp_y_test, xghyp_y_test_pred, labels=[1, 0]).ravel()  # Ensure labels are ordered as [1, 0] for TP, FN, FP, TN

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(xghyp_conf_matrix, annot=True, fmt='g', cmap='Blues',
            xticklabels=['Predicted Positive (1)', 'Predicted Negative (0)'],
            yticklabels=['Actual Positive (1)', 'Actual Negative (0)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Coronary Heart Disease or Myocardial Infarction Occurrence: XGBoost Model')
plt.show()

**Analysis:**
* True Positives (TP): 22388 instances where the model correctly predicted the presence of the disease.
* True Negatives (TN): 5849 instances where the model correctly predicted the absence of the disease.
* False Positives (FP): 2101 instances where the model incorrectly predicted the presence of the disease.
* False Negatives (FN): 57685 instances where the model incorrectly predicted the absence of the disease.

**Implication on Metrics:**
* The model seems to have a low sensitivity/recall as it misses a lot of true cases (high FN).
* The model's specificity is better in this case, indicating it's relatively more accurate when predicting the absence of the disease.
* Precision (the proportion of positive identifications that were actually correct) seems moderate but is affected by a relatively high number of false positives.
* The high number of false negatives is particularly concerning, suggesting that many individuals with the disease may not be identified by the model, which could have serious implications in a healthcare setting.

**Conclusion on both (hypertuned) LogReg and XGBoost Models:**

 It's normal and expected for scores to change - in this case, drop - as the model's focus shifts due to parameter tuning. The goal of tuning is often to improve *generalization*, not necessarily to increase the raw train-test accuracy score.

Hence, performance on specific aspects (like the ROC AUC score should be considered. The ROC AUC score is particularly useful in evaluating models on imbalanced datasets -such as in our case - because it considers both the positive and negative classes through the entire range of classification thresholds.

* ROC score LogReg:
* ROC score XGBoost:

---
#### 4.4.3 Deep Learning Modeling (with Keras)

Beyond the traditional types of modeling, we will also explore a Feedforward Neural Network (FNN), specifically designed for binary classification tasks, to compare with our hypertuned "traditional" models. This type of network is also commonly referred to as a Multilayer Perceptron (MLP) when it includes one or more hidden layers, as is the case here.

In the case here, where we're determining the predictability of a binary classification task,(having heart disease/myocardial infarction or not), we will adopt `binary crossentropy` as the loss function. This loss function is designed for two-class classification problems.

As for the activation function, the `sigmoid` is appropriate for binary classification tasks when using neural networks. It will output a value between 0 and 1, which is typically interpreted as the probability of belonging to the positive class.

* Performance Metrics: Train-Test Accuracy Score is will be used to compare against the traditional models, to evaluate the model's performance, indicating how often the model correctly classifies the input data.

##### 4.4.3.1 Binary Crossentropy

In [ ]:
pip install tensorflow keras scikit-learn

In [ ]:
# Separate features and target from the cleaned dataframe
fnn_X = df.drop('chd_mi', axis=1)
fnn_y = df['chd_mi'].astype(int).map({1: 0, 2: 1})  # Map values and ensure int type

# # Encoding categorical variables
# loghyp_X_encoded = pd.get_dummies(loghyp_X)

# # Ensuring loghyp_X and loghyp_y have consistent lengths
# assert len(loghyp_X_encoded) == len(loghyp_y), "loghyp_X and loghyp_y have inconsistent number of samples."

# Perform the train-test split
fnn_X_train, fnn_X_test, fnn_y_train, fnn_y_test = train_test_split(fnn_X, fnn_y, test_size=0.2, stratify=fnn_y, random_state=42)

# Feature scaling
scaler = StandardScaler()
fnn_X_train_scaled = scaler.fit_transform(fnn_X_train)
fnn_X_test_scaled = scaler.transform(fnn_X_test)

In [ ]:
# Define the Keras model with added dropout for regularization
model = Sequential()
model.add(Dense(64, input_dim=fnn_X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model with a potentially more effective optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping and learning rate reduction on plateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
]

# Fit the model with a larger batch size to speed up training
history = model.fit(
    fnn_X_train_scaled, fnn_y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=128,  # Larger batch size to speed up training
    verbose=1,
    callbacks=callbacks  # Use callbacks for early stopping and LR reduction
)

# Predict probabilities for the test set (ROC AUC Score)
fnn_y_test_proba = model.predict(fnn_X_test_scaled, verbose=0)

# Evaluate the model (Train-Test Score, ROC AUC Score)
fnn_train_acc = model.evaluate(fnn_X_train_scaled, fnn_y_train, verbose=0)
fnn_test_acc = model.evaluate(fnn_X_test_scaled, fnn_y_test, verbose=0)
fnn_roc_auc = roc_auc_score(fnn_y_test, fnn_y_test_proba)
fnn_f1 = f1_score(fnn_y_test, fnn_y_test_proba)
print(f'FNN Train Accuracy: {fnn_train_acc * 100:.2f}%')
print(f'FNN Test Accuracy: {fnn_test_acc * 100:.2f}%')
print(f'FNN ROC AUC Score: {fnn_roc_auc:.4f}')
print(f'FNN F1 Score: {fnn_f1:.4f}')

In [ ]:
model.summary()

In [ ]:
#Plot accuracy or loss over epochs
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('FNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train Accuracy', 'Validation Accuracy'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('FNN Model Loss', size=17)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Predict probabilities for the test set (ROC AUC Score)
fnn_y_test_proba = model.predict(fnn_X_test_scaled, verbose=0)

# Convert probabilities to binary predictions
# You need to choose a threshold for classification, usually 0.5 for binary classification
threshold = 0.5
fnn_y_test_pred = np.where(fnn_y_test_proba.flatten() > threshold, 1, 0)

# Compute the confusion matrix using the true labels (y_test) and predicted labels (y_test_pred)
fnn_conf_matrix = confusion_matrix(fnn_y_test, fnn_y_test_pred)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(fnn_conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted Negative (0)', 'Predicted Positive (1)'],
            yticklabels=['Actual Negative (0)', 'Actual Positive (1)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix: FNN Model')
plt.show()

asda

---

### 4.5 Conclusion

---

#### 4.5.1 Summary


#### 4.5.2 Limitations

The models w

Limitations of Logistic Regression:

Assumption of Linearity: Logistic Regression assumes a linear relationship between the independent variables and the log odds of the dependent variable. This can limit its ability to model more complex, non-linear relationships that might exist in the data.
Feature Importance: It may not perform well with a large number of categorical features or variables that are interrelated (multicollinearity). Feature scaling and selection become critical.
Binary Outcomes: It is primarily used for binary classification problems. Although it can be extended to multiclass classification, it may not perform as well as inherently multiclass models.

Limitations of XGBoost:

Overfitting: If not carefully tuned, XGBoost can overfit to the training data, especially when the data is noisy or when there are too many trees.
Computationally Intensive: Hyperparameter tuning for XGBoost can be computationally expensive, as it involves training multiple gradient boosting trees with cross-validation.
Interpretability: While it does offer some level of interpretability through feature importance scores, understanding how the model arrived at a prediction can be more challenging compared to simpler models like Logistic Regression.

#### 4.5.3 Recommendations